# Results in the test set: calculation of 95% confidence interval

## Import needed packages

In [1]:
import SimpleITK as sitk
from os import listdir
import numpy as np
import re
import scipy
import h5py
import math
from numpy import save
from numpy import savez_compressed
from numpy import load
from volumentations import *
from random import randrange
from sklearn.utils import shuffle
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import activations
from tensorflow.keras import initializers
from tensorflow.keras import backend as K
import gc
from tensorflow.keras.utils import Sequence
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.models import load_model
from sklearn.metrics import fbeta_score
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from scipy import stats
from scipy import ndimage
from patchify import patchify, unpatchify
import segmentation_models_3D as sm
import matplotlib.gridspec as gridspec
import nibabel as nb

2025-09-07 09:48:34.996912: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-09-07 09:48:35.013855: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-09-07 09:48:35.034028: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-09-07 09:48:35.039827: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-09-07 09:48:35.054232: I tensorflow/core/platform/cpu_feature_guar

Segmentation Models: using `keras` framework.


## Ingest data in the test set

In [2]:
# Paths to the test directory with original MRIs and segmentations
MRIs_test_dir = '/home/ch134560/data/Test/'

# Create list with all the MRI files
MRI_test_files = [MRIs_test_dir + MRI_test_file for MRI_test_file in listdir(MRIs_test_dir)]
MRI_test_files.sort()


## Iterate through the MRIs 
# Initiliaze values
current_case = re.search('case(\d+)', MRI_test_files[0])
if current_case: 
    current_case = current_case.group()
else:
    current_case = 'case missing'
current_scan = re.search('s(\d{2})', MRI_test_files[0])
if current_scan: 
    current_scan = current_scan.group()
else:
    current_scan = 'scan missing'
T1 = 0
T2 = 0
flair = 0
segmentation = 0
case_files = [None, None, None, None]
included_test_files = []

# Iterate through the MRIs 
for test_MRI in MRI_test_files:   
    # Define the case
    case = re.search('case(\d+)', test_MRI)
    if case: 
        case = case.group()
    else:
        case = 'case missing'
    scan = re.search('s(\d{2})', test_MRI)
    if scan:
        scan = scan.group()
    else:
        scan = 'scan missing'
    # Update the current case if case is new
    if case != current_case:
        current_case = case
        T1 = 0
        T2 = 0
        flair = 0
        segmentation = 0
        case_files = [None, None, None, None]
    if scan != current_scan:
        current_scan = scan
        T1 = 0
        T2 = 0
        flair = 0
        segmentation = 0     
        case_files = [None, None, None, None]
        
    # Include only cases with a T1, T2, FLAIR, and a segmentation
    if ('FLAIR' not in test_MRI) and ('lesion' not in test_MRI) and ('t2w' not in test_MRI) and T1==0:
        case_files[0] = test_MRI
        T1 += 1    
    if ('t2w' in test_MRI) and T2==0:
        case_files[1] = test_MRI
        T2 += 1    
    if ('FLAIR' in test_MRI) and flair==0:
        case_files[2] = test_MRI
        flair += 1
    if ('lesion' in test_MRI) and segmentation==0:
        case_files[3] = test_MRI
        segmentation += 1
    if T1 == 1 and T2 == 1 and flair == 1 and segmentation == 1 and len(case_files) == 4:
        included_test_files = included_test_files + [tuple(case_files)]
        T1 = 0
        T2 = 0
        flair = 0
        segmentation = 0
        case_files = [None, None, None, None]

<>:11: SyntaxWarning: invalid escape sequence '\d'
<>:16: SyntaxWarning: invalid escape sequence '\d'
<>:31: SyntaxWarning: invalid escape sequence '\d'
<>:36: SyntaxWarning: invalid escape sequence '\d'
<>:11: SyntaxWarning: invalid escape sequence '\d'
<>:16: SyntaxWarning: invalid escape sequence '\d'
<>:31: SyntaxWarning: invalid escape sequence '\d'
<>:36: SyntaxWarning: invalid escape sequence '\d'
/tmp/ipykernel_388197/1046949074.py:11: SyntaxWarning: invalid escape sequence '\d'
  current_case = re.search('case(\d+)', MRI_test_files[0])
/tmp/ipykernel_388197/1046949074.py:16: SyntaxWarning: invalid escape sequence '\d'
  current_scan = re.search('s(\d{2})', MRI_test_files[0])
/tmp/ipykernel_388197/1046949074.py:31: SyntaxWarning: invalid escape sequence '\d'
  case = re.search('case(\d+)', test_MRI)
/tmp/ipykernel_388197/1046949074.py:36: SyntaxWarning: invalid escape sequence '\d'
  scan = re.search('s(\d{2})', test_MRI)


In [3]:
# Initialize the X_test and Y_test tensors
n_all_test = len(included_test_files) * 64
n_channel_test = 3
n_class_test = 2

# Initial images are 256x256x256, but they are too big for the CNN, so they are patchified into 64x64x64 patches
SX_img, SY_img, SZ_img= 256, 256, 256
SX, SY, SZ= 64, 64, 64
X_test = np.zeros( (n_all_test, SX, SY, SZ, n_channel_test), np.float32)
Y_test = np.zeros( (n_all_test, SX, SY, SZ, 1), np.uint8 )

In [4]:
# Iterate over the dataset to create the entire X_test and Y_test tensors
i = 0

for MRI_T1_file, MRI_T2_file, MRI_FLAIR_file, segmentation_file in included_test_files:

    # Read the .nii image containing a brain volume with SimpleITK and get 
    # the numpy array:
    MRI_T1_img = sitk.ReadImage(MRI_T1_file)
    MRI_T1_img = sitk.GetArrayFromImage(MRI_T1_img)

    # Normalize the image between 0 and 1:
    MRI_T1_img_max = np.max(MRI_T1_img)
    MRI_T1_img_min = np.min(MRI_T1_img)
    MRI_T1_img = (np.float32(MRI_T1_img) - np.float32(MRI_T1_img_min)) / (np.float32(MRI_T1_img_max) - np.float32(MRI_T1_img_min))
        
    # Resize for consistency
    MRI_T1_img = np.float32(np.transpose(MRI_T1_img, [2, 1, 0]))
    MRI_T1_img = scipy.ndimage.zoom(MRI_T1_img, zoom=(SX_img/MRI_T1_img.shape[0], SY_img/MRI_T1_img.shape[1], SZ_img/MRI_T1_img.shape[2]), order=0)

    # Patchify and introduce the MRI into the X tensor
    MRI_T1_img_patches = patchify(MRI_T1_img, (64, 64, 64), step=64)
    MRI_T1_img_patches = np.reshape(MRI_T1_img_patches, (-1, MRI_T1_img_patches.shape[3], MRI_T1_img_patches.shape[4], MRI_T1_img_patches.shape[5]))
    X_test[i:i+64, :, :, :, 0] = MRI_T1_img_patches 


    
    # Read the .nii image containing a brain volume with SimpleITK and get 
    # the numpy array:
    MRI_T2_img = sitk.ReadImage(MRI_T2_file)
    MRI_T2_img = sitk.GetArrayFromImage(MRI_T2_img)

    # Normalize the image between 0 and 1:
    MRI_T2_img_max = np.max(MRI_T2_img)
    MRI_T2_img_min = np.min(MRI_T2_img)
    MRI_T2_img = (np.float32(MRI_T2_img) - np.float32(MRI_T2_img_min)) / (np.float32(MRI_T2_img_max) - np.float32(MRI_T2_img_min))
        
    # Resize for consistency
    MRI_T2_img = np.float32(np.transpose(MRI_T2_img, [2, 1, 0]))
    MRI_T2_img = scipy.ndimage.zoom(MRI_T2_img, zoom=(SX_img/MRI_T2_img.shape[0], SY_img/MRI_T2_img.shape[1], SZ_img/MRI_T2_img.shape[2]), order=0)

    # Patchify and introduce the MRI into the X tensor
    MRI_T2_img_patches = patchify(MRI_T2_img, (64, 64, 64), step=64)
    MRI_T2_img_patches = np.reshape(MRI_T2_img_patches, (-1, MRI_T2_img_patches.shape[3], MRI_T2_img_patches.shape[4], MRI_T2_img_patches.shape[5]))
    X_test[i:i+64, :, :, :, 1] = MRI_T2_img_patches 

    
    
    # Read the .nii image containing a brain volume with SimpleITK and get 
    # the numpy array:
    MRI_FLAIR_img = sitk.ReadImage(MRI_FLAIR_file)
    MRI_FLAIR_img = sitk.GetArrayFromImage(MRI_FLAIR_img)

    # Normalize the image between 0 and 1:
    MRI_FLAIR_img_max = np.max(MRI_FLAIR_img)
    MRI_FLAIR_img_min = np.min(MRI_FLAIR_img)
    MRI_FLAIR_img = (np.float32(MRI_FLAIR_img) - np.float32(MRI_FLAIR_img_min)) / (np.float32(MRI_FLAIR_img_max) - np.float32(MRI_FLAIR_img_min))
        
    # Resize for consistency
    MRI_FLAIR_img = np.float32(np.transpose(MRI_FLAIR_img, [2, 1, 0]))
    MRI_FLAIR_img = scipy.ndimage.zoom(MRI_FLAIR_img, zoom=(SX_img/MRI_FLAIR_img.shape[0], SY_img/MRI_FLAIR_img.shape[1], SZ_img/MRI_FLAIR_img.shape[2]), order=0)

    # Patchify and introduce the MRI into the X tensor
    MRI_FLAIR_img_patches = patchify(MRI_FLAIR_img, (64, 64, 64), step=64)
    MRI_FLAIR_img_patches = np.reshape(MRI_FLAIR_img_patches, (-1, MRI_FLAIR_img_patches.shape[3], MRI_FLAIR_img_patches.shape[4], MRI_FLAIR_img_patches.shape[5]))
    X_test[i:i+64, :, :, :, 2] = MRI_FLAIR_img_patches        
        


    
    # Read the .nii image containing a brain volume with SimpleITK and get 
    # the numpy array:
    segmentation_img = sitk.ReadImage(segmentation_file)
    segmentation_img = sitk.GetArrayFromImage(segmentation_img)
    
    # Do not normalize because segmentations should be 1s and 0s 

    # Resize for consistency
    segmentation_img = np.transpose(segmentation_img, [2, 1, 0])
    segmentation_img = scipy.ndimage.zoom(segmentation_img, zoom=(SX_img/segmentation_img.shape[0], SY_img/segmentation_img.shape[1], SZ_img/segmentation_img.shape[2]), order=0)

    # Patchify with a homemade method and introduce the MRI into the Y tensor
    segmentation_img_patches = patchify(segmentation_img, (64, 64, 64), step=64)
    segmentation_img_patches = np.reshape(segmentation_img_patches, (-1, segmentation_img_patches.shape[3], segmentation_img_patches.shape[4], segmentation_img_patches.shape[5]))
    Y_test[i:i+64, :, :, :, 0] = segmentation_img_patches     
    
    # Update counting value
    i += 64

In [5]:
X_test.shape, Y_test.shape

((3072, 64, 64, 64, 3), (3072, 64, 64, 64, 1))

In [6]:
# Assumed that 2s and 3s are not markers of tubers
Y_test[Y_test > 1] = 0

In [7]:
# Change the number type to float 32
Y_test = Y_test.astype(np.float32)

## Define variables

In [8]:
def dice_coef(y_true, y_pred, smooth=1e-6):
  y_true_f = K.flatten(y_true)
  y_pred_f = K.flatten(y_pred)
  intersection = K.sum(y_true_f * y_pred_f)
  return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def dice_coef_loss(y_true, y_pred):
  return 1 - dice_coef(y_true, y_pred)

## Upload the trained model

In [9]:
# Load the trained final model for prediction
trained_u_cnn = load_model('/home/ch134560/data/TSCCNN3D_dropout_augmentation_025_075_1.h5', compile=False)

2025-09-07 09:50:20.259278: I tensorflow/core/common_runtime/gpu/gpu_device.cc:2021] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 79196 MB memory:  -> device: 0, name: NVIDIA A100 80GB PCIe, pci bus id: 0000:17:00.0, compute capability: 8.0
2025-09-07 09:50:20.260820: I tensorflow/core/common_runtime/gpu/gpu_device.cc:2021] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 79196 MB memory:  -> device: 1, name: NVIDIA A100 80GB PCIe, pci bus id: 0000:31:00.0, compute capability: 8.0


In [10]:
#Predict on the test data
Y_predictions = trained_u_cnn.predict(X_test, batch_size=16)
Y_test
Y_predictions.shape, Y_test.shape

I0000 00:00:1757253030.842104  388318 service.cc:146] XLA service 0x7f63600297f0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1757253030.842166  388318 service.cc:154]   StreamExecutor device (0): NVIDIA A100 80GB PCIe, Compute Capability 8.0
I0000 00:00:1757253030.842173  388318 service.cc:154]   StreamExecutor device (1): NVIDIA A100 80GB PCIe, Compute Capability 8.0
2025-09-07 09:50:30.982230: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-09-07 09:50:31.506871: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8907


  2/192 ━━━━━━━━━━━━━━━━━━━━ 20s 107ms/step

I0000 00:00:1757253044.342458  388318 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


192/192 ━━━━━━━━━━━━━━━━━━━━ 39s 130ms/step


((3072, 64, 64, 64, 1), (3072, 64, 64, 64, 1))

In [11]:
# Define the threshold probability to call a tuber
Y_predictions[Y_predictions < 0.5] = 0.0
Y_predictions[Y_predictions >= 0.5] = 1.0

In [12]:
# Test Dice Coefficient
dice_coef(Y_test, Y_predictions)

<tf.Tensor: shape=(), dtype=float32, numpy=0.820228>

## Extract parcellation masks and calculate Dice coefficient (total and in lobes)

In [13]:
####################################GRAY MATTER MASK####################################
# Read the .nii image containing a brain volume with SimpleITK and get 
# the numpy array:
gray_matter_mask = sitk.ReadImage('/home/ch134560/data/masks/gray_matter_total_lobes_binarized.nii')
gray_matter_mask = sitk.GetArrayFromImage(gray_matter_mask)

# Resize for consistency
gray_matter_mask = np.float32(np.transpose(gray_matter_mask, [2, 1, 0]))
gray_matter_mask = scipy.ndimage.zoom(gray_matter_mask, zoom=(SX_img/gray_matter_mask.shape[0], SY_img/gray_matter_mask.shape[1], SZ_img/gray_matter_mask.shape[2]), order=0)




####################################RIGHT OCCIPITAL MASK####################################
# Read the .nii image containing a brain volume with SimpleITK and get 
# the numpy array:
R_occipital_mask = sitk.ReadImage('/home/ch134560/data/masks/mni_right_Occipital_Lobe_mask.nii.gz')
R_occipital_mask = sitk.GetArrayFromImage(R_occipital_mask)

# Resize for consistency
R_occipital_mask = np.float32(np.transpose(R_occipital_mask, [2, 1, 0]))
R_occipital_mask = scipy.ndimage.zoom(R_occipital_mask, zoom=(SX_img/R_occipital_mask.shape[0], SY_img/R_occipital_mask.shape[1], SZ_img/R_occipital_mask.shape[2]), order=0)




####################################LEFT OCCIPITAL MASK####################################
# Read the .nii image containing a brain volume with SimpleITK and get 
# the numpy array:
L_occipital_mask = sitk.ReadImage('/home/ch134560/data/masks/mni_left_Occipital_Lobe_mask.nii.gz')
L_occipital_mask = sitk.GetArrayFromImage(L_occipital_mask)

# Resize for consistency
L_occipital_mask = np.float32(np.transpose(L_occipital_mask, [2, 1, 0]))
L_occipital_mask = scipy.ndimage.zoom(L_occipital_mask, zoom=(SX_img/L_occipital_mask.shape[0], SY_img/L_occipital_mask.shape[1], SZ_img/L_occipital_mask.shape[2]), order=0)




####################################RIGHT PARIETAL MASK####################################
# Read the .nii image containing a brain volume with SimpleITK and get 
# the numpy array:
R_parietal_mask = sitk.ReadImage('/home/ch134560/data/masks/mni_right_Parietal_Lobe_mask.nii.gz')
R_parietal_mask = sitk.GetArrayFromImage(R_parietal_mask)

# Resize for consistency
R_parietal_mask = np.float32(np.transpose(R_parietal_mask, [2, 1, 0]))
R_parietal_mask = scipy.ndimage.zoom(R_parietal_mask, zoom=(SX_img/R_parietal_mask.shape[0], SY_img/R_parietal_mask.shape[1], SZ_img/R_parietal_mask.shape[2]), order=0)




####################################LEFT PARIETAL MASK####################################
# Read the .nii image containing a brain volume with SimpleITK and get 
# the numpy array:
L_parietal_mask = sitk.ReadImage('/home/ch134560/data/masks/mni_left_Parietal_Lobe_mask.nii.gz')
L_parietal_mask = sitk.GetArrayFromImage(L_parietal_mask)

# Resize for consistency
L_parietal_mask = np.float32(np.transpose(L_parietal_mask, [2, 1, 0]))
L_parietal_mask = scipy.ndimage.zoom(L_parietal_mask, zoom=(SX_img/L_parietal_mask.shape[0], SY_img/L_parietal_mask.shape[1], SZ_img/L_parietal_mask.shape[2]), order=0)




####################################RIGHT TEMPORAL MASK####################################
# Read the .nii image containing a brain volume with SimpleITK and get 
# the numpy array:
R_temporal_mask = sitk.ReadImage('/home/ch134560/data/masks/mni_right_Temporal_Lobe_mask.nii')
R_temporal_mask = sitk.GetArrayFromImage(R_temporal_mask)

# Resize for consistency
R_temporal_mask = np.float32(np.transpose(R_temporal_mask, [2, 1, 0]))
R_temporal_mask = scipy.ndimage.zoom(R_temporal_mask, zoom=(SX_img/R_temporal_mask.shape[0], SY_img/R_temporal_mask.shape[1], SZ_img/R_temporal_mask.shape[2]), order=0)




####################################LEFT TEMPORAL MASK####################################
# Read the .nii image containing a brain volume with SimpleITK and get 
# the numpy array:
L_temporal_mask = sitk.ReadImage('/home/ch134560/data/masks/mni_left_Temporal_Lobe_mask.nii')
L_temporal_mask = sitk.GetArrayFromImage(L_temporal_mask)

# Resize for consistency
L_temporal_mask = np.float32(np.transpose(L_temporal_mask, [2, 1, 0]))
L_temporal_mask = scipy.ndimage.zoom(L_temporal_mask, zoom=(SX_img/L_temporal_mask.shape[0], SY_img/L_temporal_mask.shape[1], SZ_img/L_temporal_mask.shape[2]), order=0)




####################################RIGHT FRONTAL MASK####################################
# Read the .nii image containing a brain volume with SimpleITK and get 
# the numpy array:
R_frontal_mask = sitk.ReadImage('/home/ch134560/data/masks/mni_right_Frontal_Lobe_mask.nii.gz')
R_frontal_mask = sitk.GetArrayFromImage(R_frontal_mask)

# Resize for consistency
R_frontal_mask = np.float32(np.transpose(R_frontal_mask, [2, 1, 0]))
R_frontal_mask = scipy.ndimage.zoom(R_frontal_mask, zoom=(SX_img/R_frontal_mask.shape[0], SY_img/R_frontal_mask.shape[1], SZ_img/R_frontal_mask.shape[2]), order=0)




####################################LEFT FRONTAL MASK####################################
# Read the .nii image containing a brain volume with SimpleITK and get 
# the numpy array:
L_frontal_mask = sitk.ReadImage('/home/ch134560/data/masks/mni_left_Frontal_Lobe_mask.nii.gz')
L_frontal_mask = sitk.GetArrayFromImage(L_frontal_mask)

# Resize for consistency
L_frontal_mask = np.float32(np.transpose(L_frontal_mask, [2, 1, 0]))
L_frontal_mask = scipy.ndimage.zoom(L_frontal_mask, zoom=(SX_img/L_frontal_mask.shape[0], SY_img/L_frontal_mask.shape[1], SZ_img/L_frontal_mask.shape[2]), order=0)

#### Dice coefficient: gray matter

In [14]:
# Initialize the tensor where the reformatted masks will be saved
Y_gray_matter_mask = np.zeros( (n_all_test, SX, SY, SZ, 1), np.uint8 )

# Patchify and replicate the mask to fit the shape of the Y_test and Y_validation tensors
for i in range(len(included_test_files)):
    # Patchify with a homemade method and introduce the MRI into the Y tensor
    gray_matter_mask_patches = patchify(gray_matter_mask, (64, 64, 64), step=64)
    gray_matter_mask_patches = np.reshape(gray_matter_mask_patches, (-1, gray_matter_mask_patches.shape[3], gray_matter_mask_patches.shape[4], gray_matter_mask_patches.shape[5]))
    Y_gray_matter_mask[i*64:i*64+64, :, :, :, 0] = gray_matter_mask_patches    

Y_gray_matter_mask = Y_gray_matter_mask.astype(np.float32)

In [15]:
# Mask the Y_test and Y_predictions
Y_test_gray_matter = Y_test * Y_gray_matter_mask
Y_predictions_gray_matter = Y_predictions * Y_gray_matter_mask

In [16]:
# Test Dice Coefficient: gray matter
dice_coef(Y_test_gray_matter, Y_predictions_gray_matter)

<tf.Tensor: shape=(), dtype=float32, numpy=0.8304205>

#### Dice coefficient: left frontal

In [17]:
# Initialize the tensor where the reformatted masks will be saved
Y_L_frontal_mask = np.zeros( (n_all_test, SX, SY, SZ, 1), np.uint8 )

# Patchify and replicate the mask to fit the shape of the Y_test and Y_validation tensors
for i in range(len(included_test_files)):
    # Patchify with a homemade method and introduce the MRI into the Y tensor
    L_frontal_mask_patches = patchify(L_frontal_mask, (64, 64, 64), step=64)
    L_frontal_mask_patches = np.reshape(L_frontal_mask_patches, (-1, L_frontal_mask_patches.shape[3], L_frontal_mask_patches.shape[4], L_frontal_mask_patches.shape[5]))
    Y_L_frontal_mask[i*64:i*64+64, :, :, :, 0] = L_frontal_mask_patches    

Y_L_frontal_mask = Y_L_frontal_mask.astype(np.float32)

In [18]:
# Mask the Y_test and Y_predictions
Y_test_L_frontal = Y_test * Y_L_frontal_mask
Y_predictions_L_frontal = Y_predictions * Y_L_frontal_mask

In [19]:
# Test Dice Coefficient: left frontal
dice_coef(Y_test_L_frontal, Y_predictions_L_frontal)

<tf.Tensor: shape=(), dtype=float32, numpy=0.8308467>

#### Dice coefficient: right frontal

In [20]:
# Initialize the tensor where the reformatted masks will be saved
Y_R_frontal_mask = np.zeros( (n_all_test, SX, SY, SZ, 1), np.uint8 )

# Patchify and replicate the mask to fit the shape of the Y_test and Y_validation tensors
for i in range(len(included_test_files)):
    # Patchify with a homemade method and introduce the MRI into the Y tensor
    R_frontal_mask_patches = patchify(R_frontal_mask, (64, 64, 64), step=64)
    R_frontal_mask_patches = np.reshape(R_frontal_mask_patches, (-1, R_frontal_mask_patches.shape[3], R_frontal_mask_patches.shape[4], R_frontal_mask_patches.shape[5]))
    Y_R_frontal_mask[i*64:i*64+64, :, :, :, 0] = R_frontal_mask_patches    

Y_R_frontal_mask = Y_R_frontal_mask.astype(np.float32)

In [21]:
# Mask the Y_test and Y_predictions
Y_test_R_frontal = Y_test * Y_R_frontal_mask
Y_predictions_R_frontal = Y_predictions * Y_R_frontal_mask

In [22]:
# Test Dice Coefficient: right frontal
dice_coef(Y_test_R_frontal, Y_predictions_R_frontal)

<tf.Tensor: shape=(), dtype=float32, numpy=0.8264952>

#### Dice coefficient: left temporal

In [23]:
# Initialize the tensor where the reformatted masks will be saved
Y_L_temporal_mask = np.zeros( (n_all_test, SX, SY, SZ, 1), np.uint8 )

# Patchify and replicate the mask to fit the shape of the Y_test and Y_validation tensors
for i in range(len(included_test_files)):
    # Patchify with a homemade method and introduce the MRI into the Y tensor
    L_temporal_mask_patches = patchify(L_temporal_mask, (64, 64, 64), step=64)
    L_temporal_mask_patches = np.reshape(L_temporal_mask_patches, (-1, L_temporal_mask_patches.shape[3], L_temporal_mask_patches.shape[4], L_temporal_mask_patches.shape[5]))
    Y_L_temporal_mask[i*64:i*64+64, :, :, :, 0] = L_temporal_mask_patches    

Y_L_temporal_mask = Y_L_temporal_mask.astype(np.float32)

In [24]:
# Mask the Y_test and Y_predictions
Y_test_L_temporal = Y_test * Y_L_temporal_mask
Y_predictions_L_temporal = Y_predictions * Y_L_temporal_mask

In [25]:
# Test Dice Coefficient: left temporal
dice_coef(Y_test_L_temporal, Y_predictions_L_temporal)

<tf.Tensor: shape=(), dtype=float32, numpy=0.81659526>

#### Dice coefficient: right temporal

In [26]:
# Initialize the tensor where the reformatted masks will be saved
Y_R_temporal_mask = np.zeros( (n_all_test, SX, SY, SZ, 1), np.uint8 )

# Patchify and replicate the mask to fit the shape of the Y_test and Y_validation tensors
for i in range(len(included_test_files)):
    # Patchify with a homemade method and introduce the MRI into the Y tensor
    R_temporal_mask_patches = patchify(R_temporal_mask, (64, 64, 64), step=64)
    R_temporal_mask_patches = np.reshape(R_temporal_mask_patches, (-1, R_temporal_mask_patches.shape[3], R_temporal_mask_patches.shape[4], R_temporal_mask_patches.shape[5]))
    Y_R_temporal_mask[i*64:i*64+64, :, :, :, 0] = R_temporal_mask_patches    

Y_R_temporal_mask = Y_R_temporal_mask.astype(np.float32)

In [27]:
# Mask the Y_test and Y_predictions
Y_test_R_temporal = Y_test * Y_R_temporal_mask
Y_predictions_R_temporal = Y_predictions * Y_R_temporal_mask

In [28]:
# Test Dice Coefficient: right temporal
dice_coef(Y_test_R_temporal, Y_predictions_R_temporal)

<tf.Tensor: shape=(), dtype=float32, numpy=0.8344379>

#### Dice coefficient: left parietal

In [29]:
# Initialize the tensor where the reformatted masks will be saved
Y_L_parietal_mask = np.zeros( (n_all_test, SX, SY, SZ, 1), np.uint8 )

# Patchify and replicate the mask to fit the shape of the Y_test and Y_validation tensors
for i in range(len(included_test_files)):
    # Patchify with a homemade method and introduce the MRI into the Y tensor
    L_parietal_mask_patches = patchify(L_parietal_mask, (64, 64, 64), step=64)
    L_parietal_mask_patches = np.reshape(L_parietal_mask_patches, (-1, L_parietal_mask_patches.shape[3], L_parietal_mask_patches.shape[4], L_parietal_mask_patches.shape[5]))
    Y_L_parietal_mask[i*64:i*64+64, :, :, :, 0] = L_parietal_mask_patches    

Y_L_parietal_mask = Y_L_parietal_mask.astype(np.float32)

In [30]:
# Mask the Y_test and Y_predictions
Y_test_L_parietal = Y_test * Y_L_parietal_mask
Y_predictions_L_parietal = Y_predictions * Y_L_parietal_mask

In [31]:
# Test Dice Coefficient: left parietal
dice_coef(Y_test_L_parietal, Y_predictions_L_parietal)

<tf.Tensor: shape=(), dtype=float32, numpy=0.82076263>

#### Dice coefficient: right parietal

In [32]:
# Initialize the tensor where the reformatted masks will be saved
Y_R_parietal_mask = np.zeros( (n_all_test, SX, SY, SZ, 1), np.uint8 )

# Patchify and replicate the mask to fit the shape of the Y_test and Y_validation tensors
for i in range(len(included_test_files)):
    # Patchify with a homemade method and introduce the MRI into the Y tensor
    R_parietal_mask_patches = patchify(R_parietal_mask, (64, 64, 64), step=64)
    R_parietal_mask_patches = np.reshape(R_parietal_mask_patches, (-1, R_parietal_mask_patches.shape[3], R_parietal_mask_patches.shape[4], R_parietal_mask_patches.shape[5]))
    Y_R_parietal_mask[i*64:i*64+64, :, :, :, 0] = R_parietal_mask_patches    

Y_R_parietal_mask = Y_R_parietal_mask.astype(np.float32)

In [33]:
# Mask the Y_test and Y_predictions
Y_test_R_parietal = Y_test * Y_R_parietal_mask
Y_predictions_R_parietal = Y_predictions * Y_R_parietal_mask

In [34]:
# Test Dice Coefficient: right temporal
dice_coef(Y_test_R_parietal, Y_predictions_R_parietal)

<tf.Tensor: shape=(), dtype=float32, numpy=0.84015995>

#### Dice coefficient: left occipital

In [35]:
# Initialize the tensor where the reformatted masks will be saved
Y_L_occipital_mask = np.zeros( (n_all_test, SX, SY, SZ, 1), np.uint8 )

# Patchify and replicate the mask to fit the shape of the Y_test and Y_validation tensors
for i in range(len(included_test_files)):
    # Patchify with a homemade method and introduce the MRI into the Y tensor
    L_occipital_mask_patches = patchify(L_occipital_mask, (64, 64, 64), step=64)
    L_occipital_mask_patches = np.reshape(L_occipital_mask_patches, (-1, L_occipital_mask_patches.shape[3], L_occipital_mask_patches.shape[4], L_occipital_mask_patches.shape[5]))
    Y_L_occipital_mask[i*64:i*64+64, :, :, :, 0] = L_occipital_mask_patches    

Y_L_occipital_mask = Y_L_occipital_mask.astype(np.float32)

In [36]:
# Mask the Y_test and Y_predictions
Y_test_L_occipital = Y_test * Y_L_occipital_mask
Y_predictions_L_occipital = Y_predictions * Y_L_occipital_mask

In [37]:
# Test Dice Coefficient: left occipital
dice_coef(Y_test_L_occipital, Y_predictions_L_occipital)

<tf.Tensor: shape=(), dtype=float32, numpy=0.8315069>

#### Dice coefficient: right occipital

In [38]:
# Initialize the tensor where the reformatted masks will be saved
Y_R_occipital_mask = np.zeros( (n_all_test, SX, SY, SZ, 1), np.uint8 )

# Patchify and replicate the mask to fit the shape of the Y_test and Y_validation tensors
for i in range(len(included_test_files)):
    # Patchify with a homemade method and introduce the MRI into the Y tensor
    R_occipital_mask_patches = patchify(R_occipital_mask, (64, 64, 64), step=64)
    R_occipital_mask_patches = np.reshape(R_occipital_mask_patches, (-1, R_occipital_mask_patches.shape[3], R_occipital_mask_patches.shape[4], R_occipital_mask_patches.shape[5]))
    Y_R_occipital_mask[i*64:i*64+64, :, :, :, 0] = R_occipital_mask_patches    

Y_R_occipital_mask = Y_R_occipital_mask.astype(np.float32)

In [39]:
# Mask the Y_test and Y_predictions
Y_test_R_occipital = Y_test * Y_R_occipital_mask
Y_predictions_R_occipital = Y_predictions * Y_R_occipital_mask

In [40]:
# Test Dice Coefficient: right occipital
dice_coef(Y_test_R_occipital, Y_predictions_R_occipital)

<tf.Tensor: shape=(), dtype=float32, numpy=0.8554715>

## Create the bootstrap to calculate the 95% confidence interval

In [41]:
# Create an empty list to save the different Dice coefficients of each run
DSSCs = []
DSSCs_gray_matter = []
DSSCs_L_frontal = []
DSSCs_R_frontal = []
DSSCs_L_temporal = []
DSSCs_R_temporal = []
DSSCs_L_parietal = []
DSSCs_R_parietal = []
DSSCs_L_occipital = []
DSSCs_R_occipital = []


# Create a loop in which each time we sample with replacement from the 48 MRIs in the test set. 1000 iterations with replacement
for j in range(1000):
    indices = randint(0, 48, 48)

    # Initialize the X_test and Y_test tensors
    n_all_test = len(included_test_files) * 64
    n_channel_test = 3
    n_class_test = 2
    
    # Initial images are 256x256x256, but they are too big for the CNN, so they are patchified into 64x64x64 patches
    SX_img, SY_img, SZ_img= 256, 256, 256
    SX, SY, SZ= 64, 64, 64
    X_test_sample_with_replacement = np.zeros( (n_all_test, SX, SY, SZ, n_channel_test), np.float32)
    Y_test_sample_with_replacement = np.zeros( (n_all_test, SX, SY, SZ, 1), np.uint8 )
        
    # Loop over the indices to sample with replacement
    for i,sample in enumerate(indices):
        X_test_sample_with_replacement[i*64:i*64+64, :, :, :, 0] = X_test[sample*64:sample*64+64, :, :, :, 0]
        X_test_sample_with_replacement[i*64:i*64+64, :, :, :, 1] = X_test[sample*64:sample*64+64, :, :, :, 1]
        X_test_sample_with_replacement[i*64:i*64+64, :, :, :, 2] = X_test[sample*64:sample*64+64, :, :, :, 2]
        Y_test_sample_with_replacement[i*64:i*64+64, :, :, :, 0] = Y_test[sample*64:sample*64+64, :, :, :, 0] 

    # Change the number type to float 32
    Y_test_sample_with_replacement = Y_test_sample_with_replacement.astype(np.float32)

    #Predict on the test data
    Y_predictions_sample_with_replacement = trained_u_cnn.predict(X_test_sample_with_replacement, batch_size=16)

    # Define the threshold probability to call a tuber
    Y_predictions_sample_with_replacement[Y_predictions_sample_with_replacement < 0.5] = 0.0
    Y_predictions_sample_with_replacement[Y_predictions_sample_with_replacement >= 0.5] = 1.0

    # Calculate the Dice coefficient and save it into the list
    DSSC = dice_coef(Y_test_sample_with_replacement, Y_predictions_sample_with_replacement).numpy()
    DSSCs.append(DSSC)

    # Calculate the gray matter Dice coefficient and save it into the list
    Y_test_sample_with_replacement_gray_matter = Y_test_sample_with_replacement * Y_gray_matter_mask
    Y_predictions_sample_with_replacement_gray_matter = Y_predictions_sample_with_replacement * Y_gray_matter_mask    
    DSSC_gray_matter = dice_coef(Y_test_sample_with_replacement_gray_matter, Y_predictions_sample_with_replacement_gray_matter).numpy()
    DSSCs_gray_matter.append(DSSC_gray_matter)

    # Calculate the left frontal Dice coefficient and save it into the list
    Y_test_sample_with_replacement_L_frontal = Y_test_sample_with_replacement * Y_L_frontal_mask
    Y_predictions_sample_with_replacement_L_frontal = Y_predictions_sample_with_replacement * Y_L_frontal_mask    
    DSSC_L_frontal = dice_coef(Y_test_sample_with_replacement_L_frontal, Y_predictions_sample_with_replacement_L_frontal).numpy()
    DSSCs_L_frontal.append(DSSC_L_frontal)

    # Calculate the right frontal Dice coefficient and save it into the list
    Y_test_sample_with_replacement_R_frontal = Y_test_sample_with_replacement * Y_R_frontal_mask
    Y_predictions_sample_with_replacement_R_frontal = Y_predictions_sample_with_replacement * Y_R_frontal_mask    
    DSSC_R_frontal = dice_coef(Y_test_sample_with_replacement_R_frontal, Y_predictions_sample_with_replacement_R_frontal).numpy()
    DSSCs_R_frontal.append(DSSC_R_frontal)

    # Calculate the left temporal Dice coefficient and save it into the list
    Y_test_sample_with_replacement_L_temporal = Y_test_sample_with_replacement * Y_L_temporal_mask
    Y_predictions_sample_with_replacement_L_temporal = Y_predictions_sample_with_replacement * Y_L_temporal_mask    
    DSSC_L_temporal = dice_coef(Y_test_sample_with_replacement_L_temporal, Y_predictions_sample_with_replacement_L_temporal).numpy()
    DSSCs_L_temporal.append(DSSC_L_temporal)

    # Calculate the right temporal Dice coefficient and save it into the list
    Y_test_sample_with_replacement_R_temporal = Y_test_sample_with_replacement * Y_R_temporal_mask
    Y_predictions_sample_with_replacement_R_temporal = Y_predictions_sample_with_replacement * Y_R_temporal_mask    
    DSSC_R_temporal = dice_coef(Y_test_sample_with_replacement_R_temporal, Y_predictions_sample_with_replacement_R_temporal).numpy()
    DSSCs_R_temporal.append(DSSC_R_temporal)

    # Calculate the left parietal Dice coefficient and save it into the list
    Y_test_sample_with_replacement_L_parietal = Y_test_sample_with_replacement * Y_L_parietal_mask
    Y_predictions_sample_with_replacement_L_parietal = Y_predictions_sample_with_replacement * Y_L_parietal_mask    
    DSSC_L_parietal = dice_coef(Y_test_sample_with_replacement_L_parietal, Y_predictions_sample_with_replacement_L_parietal).numpy()
    DSSCs_L_parietal.append(DSSC_L_parietal)

    # Calculate the right parietal Dice coefficient and save it into the list
    Y_test_sample_with_replacement_R_parietal = Y_test_sample_with_replacement * Y_R_parietal_mask
    Y_predictions_sample_with_replacement_R_parietal = Y_predictions_sample_with_replacement * Y_R_parietal_mask    
    DSSC_R_parietal = dice_coef(Y_test_sample_with_replacement_R_parietal, Y_predictions_sample_with_replacement_R_parietal).numpy()
    DSSCs_R_parietal.append(DSSC_R_parietal)

    # Calculate the left occipital Dice coefficient and save it into the list
    Y_test_sample_with_replacement_L_occipital = Y_test_sample_with_replacement * Y_L_occipital_mask
    Y_predictions_sample_with_replacement_L_occipital = Y_predictions_sample_with_replacement * Y_L_occipital_mask    
    DSSC_L_occipital = dice_coef(Y_test_sample_with_replacement_L_occipital, Y_predictions_sample_with_replacement_L_occipital).numpy()
    DSSCs_L_occipital.append(DSSC_L_occipital)

    # Calculate the right occipital Dice coefficient and save it into the list
    Y_test_sample_with_replacement_R_occipital = Y_test_sample_with_replacement * Y_R_occipital_mask
    Y_predictions_sample_with_replacement_R_occipital = Y_predictions_sample_with_replacement * Y_R_occipital_mask    
    DSSC_R_occipital = dice_coef(Y_test_sample_with_replacement_R_occipital, Y_predictions_sample_with_replacement_R_occipital).numpy()
    DSSCs_R_occipital.append(DSSC_R_occipital)

192/192 ━━━━━━━━━━━━━━━━━━━━ 25s 131ms/step
192/192 ━━━━━━━━━━━━━━━━━━━━ 25s 132ms/step
192/192 ━━━━━━━━━━━━━━━━━━━━ 25s 133ms/step
192/192 ━━━━━━━━━━━━━━━━━━━━ 27s 139ms/step
192/192 ━━━━━━━━━━━━━━━━━━━━ 25s 129ms/step
192/192 ━━━━━━━━━━━━━━━━━━━━ 25s 132ms/step
192/192 ━━━━━━━━━━━━━━━━━━━━ 26s 133ms/step
192/192 ━━━━━━━━━━━━━━━━━━━━ 27s 140ms/step
192/192 ━━━━━━━━━━━━━━━━━━━━ 33s 173ms/step
192/192 ━━━━━━━━━━━━━━━━━━━━ 25s 133ms/step
192/192 ━━━━━━━━━━━━━━━━━━━━ 28s 148ms/step
192/192 ━━━━━━━━━━━━━━━━━━━━ 25s 132ms/step
192/192 ━━━━━━━━━━━━━━━━━━━━ 26s 134ms/step
192/192 ━━━━━━━━━━━━━━━━━━━━ 27s 142ms/step
192/192 ━━━━━━━━━━━━━━━━━━━━ 25s 132ms/step
192/192 ━━━━━━━━━━━━━━━━━━━━ 25s 133ms/step
192/192 ━━━━━━━━━━━━━━━━━━━━ 28s 149ms/step
192/192 ━━━━━━━━━━━━━━━━━━━━ 25s 132ms/step
192/192 ━━━━━━━━━━━━━━━━━━━━ 25s 133ms/step
192/192 ━━━━━━━━━━━━━━━━━━━━ 31s 161ms/step
192/192 ━━━━━━━━━━━━━━━━━━━━ 25s 132ms/step
192/192 ━━━━━━━━━━━━━━━━━━━━ 28s 148ms/step
192/192 ━━━━━━━━━━━━━━━━━━━━ 25s

## Calculate the 95% confidence interval based on the bootstrap method

#### Whole brain

In [42]:
# Calculate the lower level of the 95% confidence interval
np.percentile(DSSCs, 2.5)

0.7993647783994675

In [43]:
# Calculate the upper level of the 95% confidence interval
np.percentile(DSSCs, 97.5)

0.8397900879383087

In [44]:
# Calculate median
np.median(DSSCs)

0.82037866

#### Gray matter

In [45]:
# Calculate the lower level of the 95% confidence interval
np.percentile(DSSCs_gray_matter, 2.5)

0.8100231841206551

In [46]:
# Calculate the upper level of the 95% confidence interval
np.percentile(DSSCs_gray_matter, 97.5)

0.8487108662724495

In [47]:
# Calculate median
np.median(DSSCs_gray_matter)

0.8306408

#### Left frontal

In [48]:
# Calculate the lower level of the 95% confidence interval
np.percentile(DSSCs_L_frontal, 2.5)

0.8042757987976075

In [49]:
# Calculate the upper level of the 95% confidence interval
np.percentile(DSSCs_L_frontal, 97.5)

0.8503992632031441

In [50]:
# Calculate median
np.median(DSSCs_L_frontal)

0.8305613

#### Right frontal

In [51]:
# Calculate the lower level of the 95% confidence interval
np.percentile(DSSCs_R_frontal, 2.5)

0.7984504476189613

In [52]:
# Calculate the upper level of the 95% confidence interval
np.percentile(DSSCs_R_frontal, 97.5)

0.8526838898658753

In [53]:
# Calculate median
np.median(DSSCs_R_frontal)

0.82769096

#### Left temporal

In [54]:
# Calculate the lower level of the 95% confidence interval
np.percentile(DSSCs_L_temporal, 2.5)

0.7790532544255256

In [55]:
# Calculate the upper level of the 95% confidence interval
np.percentile(DSSCs_L_temporal, 97.5)

0.8423813924193382

In [56]:
# Calculate median
np.median(DSSCs_L_temporal)

0.8167043

#### Right temporal

In [57]:
# Calculate the lower level of the 95% confidence interval
np.percentile(DSSCs_R_temporal, 2.5)

0.8114883229136467

In [58]:
# Calculate the upper level of the 95% confidence interval
np.percentile(DSSCs_R_temporal, 97.5)

0.8492307037115097

In [59]:
# Calculate median
np.median(DSSCs_R_temporal)

0.8342031

#### Left parietal

In [60]:
# Calculate the lower level of the 95% confidence interval
np.percentile(DSSCs_L_parietal, 2.5)

0.7824514910578728

In [61]:
# Calculate the upper level of the 95% confidence interval
np.percentile(DSSCs_L_parietal, 97.5)

0.8558406934142113

In [62]:
# Calculate median
np.median(DSSCs_L_parietal)

0.820707

#### Right parietal

In [63]:
# Calculate the lower level of the 95% confidence interval
np.percentile(DSSCs_R_parietal, 2.5)

0.8097572669386863

In [64]:
# Calculate the upper level of the 95% confidence interval
np.percentile(DSSCs_R_parietal, 97.5)

0.865171705186367

In [65]:
# Calculate median
np.median(DSSCs_R_parietal)

0.8408821

#### Left occipital

In [66]:
# Calculate the lower level of the 95% confidence interval
np.percentile(DSSCs_L_occipital, 2.5)

0.8075608596205711

In [67]:
# Calculate the upper level of the 95% confidence interval
np.percentile(DSSCs_L_occipital, 97.5)

0.8511042118072509

In [68]:
# Calculate median
np.median(DSSCs_L_occipital)

0.8315722

#### Right occipital

In [69]:
# Calculate the lower level of the 95% confidence interval
np.percentile(DSSCs_R_occipital, 2.5)

0.8381764963269234

In [70]:
# Calculate the upper level of the 95% confidence interval
np.percentile(DSSCs_R_occipital, 97.5)

0.8710747987031937

In [71]:
# Calculate median
np.median(DSSCs_R_occipital)

0.85524404